In [1]:
%matplotlib inline
import numpy as np
from __future__ import division
np.set_printoptions(linewidth=150)

In [2]:
class Board:
    def __init__(self,n=3):
        self.size=n**2
        self.board = np.zeros((self.size,self.size))
        self._clear_options()
    def _clear_options(self):
        self.options = np.empty((self.size,self.size,self.size))
        for i in range(self.size):
            for j in range(self.size):
                for k in range(self.size):
                    self.options[i][j][k] = k+1
    def view(self):
        return self.board.astype(int)
    def set_board(self,vals):
        if type(vals) == str:
            vals = [i for i in vals]
        vals = np.reshape(vals,(self.size,self.size))
        for i in range(self.size):
            self.set_row(vals[i],i+1)
    def set_row(self,vals,row):
        for j in range(self.size):
            try:
                self.board[row-1][j] = vals[j]
            except:
                self.board[row-1][j] = 0
    def set_column(self,vals,col):
        for i in range(self.size):
            try:
                self.board[i][col-1] = vals[i]
            except:
                self.board[i][col-1] = 0
    def set_val(self,val,pos):
        try:
            int(val)
        except:
            val = 0
        if type(pos) == tuple:
            self.board[pos[1]-1][pos[0]-1] = val
        else:
            self.board[(pos-1)//self.size][(pos-1)%self.size] = val
    def check_solved(self):
        return self.check_legal() and self.check_filled()
    def check_filled(self):
        return np.count_nonzero(self.board) == self.size**2
    def check_legal(self):
        return self.check_rows() and self.check_columns() and self.check_boxes()
    def check_rows(self,transpose=False):
        b2 = np.copy(self.board)
        if transpose:
            b2 = np.transpose(b2)
        for i in range(self.size):
            num_nonzero = np.count_nonzero(b2[i])
            num_unique = len(np.unique(b2[i]))
            if num_nonzero == self.size and num_unique != self.size:
                return False
            if num_nonzero != self.size and num_unique != num_nonzero + 1:
                return False
        return True
    def check_columns(self):
        return self.check_rows(transpose=True)
    def check_boxes(self):
        n = int(self.size**0.5)
        for i in range(n):
            for j in range(n):
                num_nonzero = 0
                unique = set()
                for k in range(n):
                    rk = self.board[n*i+k][n*j:n*j+n]
                    num_nonzero += np.count_nonzero(rk)
                    unique = unique.union(set(rk))
                num_unique = len(unique)
                if num_nonzero == self.size and num_unique != self.size:
                    return False
                if num_nonzero != self.size and num_unique != num_nonzero + 1:
                    return False
        return True
    def solve(self,brute=False):
        b2 = np.copy(self.board)
        self._clear_options()
        o2 = np.copy(self.options)
        if brute:
            success = self._brute_solver()
        else:
            success = self._smart_solver()
        if not success:
            self.board = b2
            self.options = o2
            print("Solve failed! Board is impossible to solve.")
        else:
            print("Solve complete!")
        return self.view()
    def _smart_solver(self):
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] != 0:
                    self._prune_current_options(self.board[i][j],i,j)
        found = True
        while found:
            found = self._forced_placement()
        return self._brute_solver()
    def _forced_placement(self):
        found = False
        for i in range(self.size):
            for j in range(self.size):
                if self.board[i][j] == 0 and np.count_nonzero(self.options[i][j]) == 1:
                    found = True
                    self.board[i][j] = self.options[i][j][self.options[i][j]>0][0]
#                     print(j+1,i+1,self.board[i][j])
                    self._prune_current_options(self.board[i][j],i,j)
        return found
    def _prune_current_options(self,num,i,j):
        n = int(self.size**0.5)
        self.options[i][j] = [num-1 if k == num-1 else 0 for k in range(self.size)]
        for q in range(self.size):
            self.options[q][j][num-1] = 0
        for q in range(self.size):
            self.options[i][q][num-1] = 0
        for q in range(n):
            for r in range(n):
                self.options[n*(i//n)+q][n*(j//n)+r][num-1] = 0
    def _brute_solver(self):
        if self.check_solved():
            return True
        if not self.check_legal():
            return False
        for pos in range(self.size**2):
            if self.board[pos//self.size][pos%self.size] == 0:
                break
        vals = [i for i in self.options[pos//self.size][pos%self.size] if i != 0]
        for val in vals:
            self.board[pos//self.size][pos%self.size] = val
#             print('adding ' + str(val) + ' at pos:' + str(((pos+1)%9,(pos+1)//9)))
            if self._brute_solver():
                return True
        self.board[pos//self.size][pos%self.size] = 0
        return False

In [3]:
bd = Board()

##Not legal board, but ok on rows
# for i in range(9):
#     options[i] = options[i] + i + 1
#     for j in range(9):
#         board[j][i] = board[j][i] + i + 1

##Not legal board, but ok on boxes
# for i in range(3):
#     for j in range(3):
#         board[3*i][3*j:3*j+3] = [1,2,3]
#         board[3*i+1][3*j:3*j+3] = [4,5,6]
#         board[3*i+2][3*j:3*j+3] = [7,8,9]

##Legal Board
# for i in range(3):
#     for j in range(9):
#         board[i+0][j] = (3*i+j+0)%9 + 1
#         board[i+3][j] = (3*i+j+1)%9 + 1
#         board[i+6][j] = (3*i+j+2)%9 + 1

###Here, you can create a board, set values, and view the board.

In [4]:
bd = Board()
# bd.set_val(8,36)
bd.set_val(4,(2,3))
bd.set_val(3,(7,2))
bd.set_val(2,(8,2))
bd.set_row('12345678.',1)
# bd.set_column([9,8,7,6,5,4,3,2,1],9)
# bd.set_board([i%9 for i in range(81)])
print(bd.check_legal())
bd.view()

True


array([[1, 2, 3, 4, 5, 6, 7, 8, 0],
       [0, 0, 0, 0, 0, 0, 3, 2, 0],
       [0, 4, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

###Solving is done first by finding "forced" squares (where only 1 out of 9 options is possible), then with a brute-force, recursive algorithm (sequentially trying each value in each unfilled square). 
It is reasonably fast for many boards :-)

In [5]:
%time bd.solve()

Solve complete!
Wall time: 520 ms


array([[1, 2, 3, 4, 5, 6, 7, 8, 9],
       [5, 6, 7, 1, 8, 9, 3, 2, 4],
       [8, 4, 9, 2, 3, 7, 1, 5, 6],
       [2, 1, 4, 3, 6, 5, 8, 9, 7],
       [3, 5, 8, 7, 9, 1, 4, 6, 2],
       [7, 9, 6, 8, 2, 4, 5, 1, 3],
       [4, 3, 2, 6, 1, 8, 9, 7, 5],
       [6, 8, 5, 9, 7, 3, 2, 4, 1],
       [9, 7, 1, 5, 4, 2, 6, 3, 8]])

###Enjoy! Use this notebook to play with Sudoku boards or solve puzzles that you're stuck on. (Or, check if your guesses are on the right track!)
For example, here is a [list](http://www.puzzles.ca/sudoku.html) of Sudoku puzzles.

In [6]:
##v1 is Easy Puzzle   #217 from http://www.puzzles.ca/sudoku.html
##v2 is Medium Puzzle #029 from http://www.puzzles.ca/sudoku.html
##v2 is Hard Puzzle   #001 from http://www.puzzles.ca/sudoku.html
##Any nonzero character is interpreted as "0" (unknown)
v1 =    '000090010300010200000400067' +\
        '007500000006029081509000000' +\
        '200000000008000000160305002'
v2 =    '903020806000300402006085710' +\
        '000802001600000004032000070' +\
        '010000007408050000000431000'
v3 =    '438760102200090530000002608' +\
        '004023050300000800600000000' +\
        '005010309010000080900600070'
print(len(v1), type(v1))
print(len(v2), type(v2))
print(len(v3), type(v3))

(81, <type 'str'>)
(81, <type 'str'>)
(81, <type 'str'>)


In [7]:
bd = Board()
bd.set_board(v1)
bd.view()

array([[0, 0, 0, 0, 9, 0, 0, 1, 0],
       [3, 0, 0, 0, 1, 0, 2, 0, 0],
       [0, 0, 0, 4, 0, 0, 0, 6, 7],
       [0, 0, 7, 5, 0, 0, 0, 0, 0],
       [0, 0, 6, 0, 2, 9, 0, 8, 1],
       [5, 0, 9, 0, 0, 0, 0, 0, 0],
       [2, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 8, 0, 0, 0, 0, 0, 0],
       [1, 6, 0, 3, 0, 5, 0, 0, 2]])

In [8]:
%time bd.solve()

Solve complete!
Wall time: 15 ms


array([[6, 7, 2, 8, 9, 3, 4, 1, 5],
       [3, 4, 5, 6, 1, 7, 2, 9, 8],
       [9, 8, 1, 4, 5, 2, 3, 6, 7],
       [8, 1, 7, 5, 3, 4, 6, 2, 9],
       [4, 3, 6, 7, 2, 9, 5, 8, 1],
       [5, 2, 9, 1, 6, 8, 7, 3, 4],
       [2, 5, 3, 9, 7, 1, 8, 4, 6],
       [7, 9, 8, 2, 4, 6, 1, 5, 3],
       [1, 6, 4, 3, 8, 5, 9, 7, 2]])

In [9]:
bd.set_board(v2)
bd.view()

array([[9, 0, 3, 0, 2, 0, 8, 0, 6],
       [0, 0, 0, 3, 0, 0, 4, 0, 2],
       [0, 0, 6, 0, 8, 5, 7, 1, 0],
       [0, 0, 0, 8, 0, 2, 0, 0, 1],
       [6, 0, 0, 0, 0, 0, 0, 0, 4],
       [0, 3, 2, 0, 0, 0, 0, 7, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 7],
       [4, 0, 8, 0, 5, 0, 0, 0, 0],
       [0, 0, 0, 4, 3, 1, 0, 0, 0]])

In [10]:
%time bd.solve()

Solve complete!
Wall time: 240 ms


array([[9, 7, 3, 1, 2, 4, 8, 5, 6],
       [8, 5, 1, 3, 6, 7, 4, 9, 2],
       [2, 4, 6, 9, 8, 5, 7, 1, 3],
       [5, 9, 4, 8, 7, 2, 3, 6, 1],
       [6, 8, 7, 5, 1, 3, 9, 2, 4],
       [1, 3, 2, 6, 4, 9, 5, 7, 8],
       [3, 1, 5, 2, 9, 8, 6, 4, 7],
       [4, 2, 8, 7, 5, 6, 1, 3, 9],
       [7, 6, 9, 4, 3, 1, 2, 8, 5]])

In [11]:
bd.set_board(v3)
bd.view()

array([[4, 3, 8, 7, 6, 0, 1, 0, 2],
       [2, 0, 0, 0, 9, 0, 5, 3, 0],
       [0, 0, 0, 0, 0, 2, 6, 0, 8],
       [0, 0, 4, 0, 2, 3, 0, 5, 0],
       [3, 0, 0, 0, 0, 0, 8, 0, 0],
       [6, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 5, 0, 1, 0, 3, 0, 9],
       [0, 1, 0, 0, 0, 0, 0, 8, 0],
       [9, 0, 0, 6, 0, 0, 0, 7, 0]])

In [12]:
%time bd.solve()

Solve complete!
Wall time: 323 ms


array([[4, 3, 8, 7, 6, 5, 1, 9, 2],
       [2, 6, 1, 8, 9, 4, 5, 3, 7],
       [5, 7, 9, 1, 3, 2, 6, 4, 8],
       [1, 8, 4, 9, 2, 3, 7, 5, 6],
       [3, 9, 2, 5, 7, 6, 8, 1, 4],
       [6, 5, 7, 4, 8, 1, 9, 2, 3],
       [8, 4, 5, 2, 1, 7, 3, 6, 9],
       [7, 1, 6, 3, 4, 9, 2, 8, 5],
       [9, 2, 3, 6, 5, 8, 4, 7, 1]])

###Of course, there are much faster implementations of this code. Also, a very small number of boards (~0.05%) will take a long, long time to solve. See Peter Norvig's [writeup](http://norvig.com/sudoku.html) for more information.


In [13]:
hard1  =    '.....6....59.....82....8...' +\
            '.45........3........6..3.54' +\
            '...325..6..................'
bd.set_board(hard1)
bd.view()
##This takes a very long time!
# bd.solve()

array([[0, 0, 0, 0, 0, 6, 0, 0, 0],
       [0, 5, 9, 0, 0, 0, 0, 0, 8],
       [2, 0, 0, 0, 0, 8, 0, 0, 0],
       [0, 4, 5, 0, 0, 0, 0, 0, 0],
       [0, 0, 3, 0, 0, 0, 0, 0, 0],
       [0, 0, 6, 0, 0, 3, 0, 5, 4],
       [0, 0, 0, 3, 2, 5, 0, 0, 6],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])

###You can make boards of other sizes, too.

In [14]:
bd = Board(2)
bd.set_board('0100200002000000')
bd.view()

array([[0, 1, 0, 0],
       [2, 0, 0, 0],
       [0, 2, 0, 0],
       [0, 0, 0, 0]])

In [15]:
bd.solve()

Solve complete!


array([[3, 1, 2, 4],
       [2, 4, 1, 3],
       [4, 2, 3, 1],
       [1, 3, 4, 2]])

In [16]:
bd = Board(4)
bd.set_row('12345678090',1)
bd.set_val(6,(1,2))
bd.view()

array([[1, 2, 3, 4, 5, 6, 7, 8, 0, 9, 0, 0, 0, 0, 0, 0],
       [6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [17]:
bd.solve()

Solve complete!


array([[ 1,  2,  3,  4,  5,  6,  7,  8, 10,  9, 11, 12, 13, 14, 15, 16],
       [ 6,  5,  7,  8,  1,  2,  3,  4, 13, 14, 15, 16,  9, 10, 11, 12],
       [ 9, 10, 11, 12, 13, 14, 15, 16,  1,  2,  3,  4,  5,  6,  7,  8],
       [13, 14, 15, 16,  9, 10, 11, 12,  5,  6,  7,  8,  1,  2,  3,  4],
       [ 2,  1,  4,  3,  6,  5,  8,  7,  9, 10, 12, 11, 14, 13, 16, 15],
       [ 5,  6,  8,  7,  2,  1,  4,  3, 14, 13, 16, 15, 10,  9, 12, 11],
       [10,  9, 12, 11, 14, 13, 16, 15,  2,  1,  4,  3,  6,  5,  8,  7],
       [14, 13, 16, 15, 10,  9, 12, 11,  6,  5,  8,  7,  2,  1,  4,  3],
       [ 3,  4,  1,  2,  7,  8,  5,  6, 11, 12,  9, 10, 15, 16, 13, 14],
       [ 7,  8,  5,  6,  3,  4,  1,  2, 15, 16, 13, 14, 11, 12,  9, 10],
       [11, 12,  9, 10, 15, 16, 13, 14,  3,  4,  1,  2,  7,  8,  5,  6],
       [15, 16, 13, 14, 11, 12,  9, 10,  7,  8,  5,  6,  3,  4,  1,  2],
       [ 4,  3,  2,  1,  8,  7,  6,  5, 12, 11, 10,  9, 16, 15, 14, 13],
       [ 8,  7,  6,  5,  4,  3,  2,  1, 16, 15, 14,

###Have fun!

In [18]:
bd = Board()
bd.view()

array([[0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0]])